# 数据集创建&处理

In [1]:
# from modelarts.session import Session
# session = Session()
# session.obs.download_file(src_obs_file="obs://llddy/LUKE_mindspore/data/dev_data.npy", dst_local_dir="./dev_data.npy")

In [2]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(9957:139710232549184,MainProcess):2021-10-13-14:18:18.618.58 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(9957:139710232549184,MainProcess):2021-10-13-14:18:18.695.92 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


# 创建mindrecord

In [3]:
import numpy as np
FEATURES_FILE = "./data/dev_data1.npy"
features = np.load(FEATURES_FILE)
# print(features)

In [4]:
features

array(['{"unique_id": 1000000000, "example_index": 0, "doc_span_index": 0, "tokens": ["<s>", "\\u0120Which", "\\u0120NFL", "\\u0120team", "\\u0120represented", "\\u0120the", "\\u0120AFC", "\\u0120at", "\\u0120Super", "\\u0120Bowl", "\\u012050", "?", "</s>", "</s>", "\\u0120Super", "\\u0120Bowl", "\\u012050", "\\u0120was", "\\u0120an", "\\u0120American", "\\u0120football", "\\u0120game", "\\u0120to", "\\u0120determine", "\\u0120the", "\\u0120champion", "\\u0120of", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120League", "\\u0120(", "NFL", ")", "\\u0120for", "\\u0120the", "\\u01202015", "\\u0120season", ".", "\\u0120The", "\\u0120American", "\\u0120Football", "\\u0120Conference", "\\u0120(", "A", "FC", ")", "\\u0120champion", "\\u0120Denver", "\\u0120Broncos", "\\u0120defeated", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120Conference", "\\u0120(", "N", "FC", ")", "\\u0120champion", "\\u0120Carolina", "\\u0120Panthers", "\\u012024", "\\u00e2\\u0122\\u0135", "10

In [5]:
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [6]:
len(list_dict[0]['word_ids'])

170

In [7]:
FEATURES_FILE = "./data/dev_data.npy"
features = np.load(FEATURES_FILE)

list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    # entity padding 1
    entity_size = len(slist["entity_position_ids"])
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"])
    temp = [[-1]*24 for i in range(24)]
    for i in range(24):
        if i < entity_size-1:
            temp[i]=(pad_entity(slist["entity_position_ids"][i], 24))


    slist["entity_position_ids"] =temp
    # entity_padding 2
    #slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    #slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 24)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [24,24]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

## 简单测试
- dev_data1.npy包含两个样本

In [8]:
# FEATURES_FILE = "./data/dev_data1.npy"
# features = np.load(FEATURES_FILE)





# SQUAD_MINDRECORD_FILE = "./data/dev_features1.mindrecord"
# list_dict = []
# for item in features:
#     dict_temp = json.loads(item)
#     list_dict.append(dict_temp)

# pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
# pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
# pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

# for slist in list_dict:
#     slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
#     slist["entity_ids"] = pad(slist["entity_ids"], 24)
#     slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
#     slist["word_ids"] = pad1(slist["word_ids"], 256)
#     slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
#     slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
#     slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
#     slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


# if os.path.exists(SQUAD_MINDRECORD_FILE):
#     os.remove(SQUAD_MINDRECORD_FILE)
#     os.remove(SQUAD_MINDRECORD_FILE + ".db")

# writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

# data_schema = {
#     "unique_id": {"type": "int32", "shape": [-1]},
#     "word_ids": {"type": "int32", "shape": [-1]},
#     "word_segment_ids": {"type": "int32", "shape": [-1]},
#     "word_attention_mask": {"type": "int32", "shape": [-1]},
#     "entity_ids": {"type": "int32", "shape": [-1]},
#     "entity_position_ids": {"type": "int32", "shape": [-1]},
#     "entity_segment_ids": {"type": "int32", "shape": [-1]},
#     "entity_attention_mask": {"type": "int32", "shape": [-1]},
#     #"start_positions": {"type": "int32", "shape": [-1]},
#     #"end_positions": {"type": "int32", "shape": [-1]}
# }
# writer.add_schema(data_schema, "it is a preprocessed squad dataset")

# data = []
# i = 0
# for item in list_dict:
#     i += 1
#     sample = {
#         "unique_id": np.array(item["unique_id"], dtype=np.int32),
#         "word_ids": np.array(item["word_ids"], dtype=np.int32),
#         "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
#         "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
#         "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
#         "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
#         "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
#         "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
#         #"start_positions": np.array(item["start_positions"], dtype=np.int32),
#         #"end_positions": np.array(item["end_positions"], dtype=np.int32),
#     }
    
#     data.append(sample)
#     #print(sample)
#     if i % 10 == 0:
#         writer.write_raw_data(data)
#         data = []

# if data:
#     writer.write_raw_data(data)

# writer.commit()

In [9]:
output_features = np.array(list_dict)

In [10]:
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 10641 samples


In [11]:
data_set = data_set.batch(2)
data_sample = next(data_set.create_dict_iterator())
data_sample

{'entity_attention_mask': Tensor(shape=[2, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 0 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[2, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 0 ... 0, 0, 0]]),
 'entity_position_ids': Tensor(shape=[2, 24, 24], dtype=Int32, value=
 [[[17, -1, -1 ... -1, -1, -1],
   [22, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   ...
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1]],
  [[62, 63, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   ...
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1],
   [-1, -1, -1 ... -1, -1, -1]]]),
 'entity_segment_ids': Tensor(shape=[2, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 'unique_id': Tensor(shape=[2, 1], dtype=Int32, value=
 [[1000002703],
  [1000005753]]),
 'word_attention_mask': Tensor(shape=[2, 256], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0],
 

In [12]:
data_sample['entity_attention_mask']

Tensor(shape=[2, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 0 ... 0, 0, 0]])

# Model

In [13]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 简单测试

In [14]:
# config = BertConfig()
# Luke_model = LukeForReadingComprehension(config)
# model = Model(Luke_model)

In [15]:
#logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)

In [16]:
#logits

# do_eval

In [17]:
!pip install tqdm
from tqdm import tqdm

In [18]:
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
def do_eval(dataset=None, load_checkpoint_path="", eval_batch_size=1):
    config = BertConfig()
    Luke_model = LukeForReadingComprehension(config)
    Luke_model.set_train(False)
    model = Model(Luke_model)
    param_dict = load_checkpoint(load_checkpoint_path)
    load_param_into_net(Luke_model, param_dict)
    output = []
    #model = Model(Luke_model)
    
    RawResult = collections.namedtuple("RawResult", ["unique_id", "start_logits", "end_logits"])
    columns_list = ["unique_id", "word_ids", "word_segment_ids", "word_attention_mask", "entity_ids", "entity_position_ids", "entity_segment_ids", "entity_attention_mask"]
    data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
    data_set = data_set.batch(eval_batch_size) # ,drop_remainder=True
    for data in tqdm(data_set.create_dict_iterator(num_epochs=1)):
        input_data = []
        for i in columns_list:
            input_data.append(data[i])

        unique_id, word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask = input_data
        #print(unique_id)
        logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)
        ids = unique_id.asnumpy()
        start = logits[0].asnumpy()
        end = logits[1].asnumpy()
    
        for i in range(len(data)):  # eval_batch_size
            unique_id = int(ids[i])
            start_logits = [float(x) for x in start[i].flat]
            end_logits = [float(x) for x in end[i].flat]
            output.append(RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))
        #print(word_ids.shape)
    return output

In [ ]:
dev_dataset = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
outputs = do_eval(dev_dataset, load_checkpoint_path = "./luke-large-qa.ckpt", eval_batch_size = 32)

264it [03:51,  1.22it/s]

In [ ]:
import numpy as np
np.save('dev_outputs.npy',outputs)

# 验证

In [ ]:
from model.eval_squad import read_squad_examples, write_predictions, SQuad_postprocess

In [ ]:
output = np.load("./dev_outputs.npy", allow_pickle=True)
eval_examples = read_squad_examples("./dataset/dev-v1.1.json", False)
all_predictions = write_predictions(eval_examples, output_features, output, 20, 30, True)
SQuad_postprocess("./dataset/dev-v1.1.json", all_predictions, output_metrics="output.json")

In [ ]:
outputs

In [ ]:
outputs[0][0]

In [ ]:
outputs[0][1]

In [ ]:
outputs[0][2]

In [ ]:
def evaluate(args, outputs, prefix=""):
    dataloader, examples, features, processor = load_examples(args, evaluate=True)
    all_results = []
    for batch in tqdm(dataloader, desc="eval"):
        model.eval()
        inputs = {k: v.to(args.device) for k, v in batch.items() if k != "example_indices"}
        with torch.no_grad():
            outputs = model(**inputs)

        for i, example_index in enumerate(batch["example_indices"]):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            start_logits, end_logits = [o[i].detach().cpu().tolist() for o in outputs]
            all_results.append(Result(unique_id, start_logits, end_logits))

    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
    if args.with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    do_lower_case = False
    if isinstance(args.tokenizer, BertTokenizer):
        do_lower_case = args.tokenizer.basic_tokenizer.do_lower_case

    write_predictions(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,
        args.with_negative,
        args.null_score_diff_threshold,
        args.tokenizer,
    )

    return evaluate_on_squad(
        SQUAD_EVAL_OPTS(
            os.path.join(args.data_dir, processor.dev_file),
            pred_file=output_prediction_file,
            na_prob_file=output_null_log_odds_file,
        )
    )

In [ ]:
output = np.load("./ori_dev_outputs.npy", allow_pickle=True)


In [ ]:
output[1][0]

In [ ]:
output[1][1]

In [ ]:
output[1][2]